# Classificador de Inadimplência

## Proposta

Criação de uma Rede Neural, afim de classificar a situação e projeção de inadimplência atuais de um empreendimento. Para isso, a Rede Neural busca correlações entre os dados afim de definir uma classificação de inadimplência.



## Bibliotecas Utilizadas

Utilizamos o TensorFlow para criação do modelo preditivo, e algumas outras bibliotecas que auxiliam da separação da massa de dados utilizada no processo de treinamento e validação do modelo.



In [1]:
!pip install --quiet sklearn

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

## Fonte de Dados

Carregamento do dataset, e separação de *80%* dos dados, para treino e *20%* para validação do modelo. Importante remover o atributo que será previsto pelo modelo, nesse caso **p_dias_atraso_category**, afim de remover qualquer tipo de enviesamento do modelo.



In [4]:
df_train, df_test = train_test_split(pd.read_csv('https://raw.githubusercontent.com/afonsir/cyrela-etl/main/data/parsed-data.csv'))

y_train = df_train.pop('p_dias_atraso_category')
y_test = df_test.pop('p_dias_atraso_category')

## Pré-processamento dos Dados

Para o modelo proposto, os dados utilizados necessitam ser apenas numéricos (inerente ao treinamento do modelo), então, se faz necessário, o pré-processamento dos dados categóricos para numéricos, sugerimos o uso de uma aplicação **Apache Spark**, pela performance e escalabilidade.


In [5]:
df_train.head()

,empresa,p_empresa,marca,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,dt_venda,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,dt_chaves,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
2805,168,0.053097,LIVING,2,4718,0.475126,1,1.0,203,0.088108,2020-09-29,0.966667,0.750000,1.0100,2023-06-01,0.033333,0.500000,1.0115,313637,0.024126,313637.49,0.024126,1.0,0.333333,1.0105,-823,0.710708,406358.11,0.036531,379533.50,0.034503,693170.99,0.033813
2046,661,0.208913,CYRELA,1,2339,0.235549,1,1.0,1305,0.566406,2020-02-29,0.966667,0.166667,1.0100,2022-12-01,0.033333,1.000000,1.0110,190869,0.014682,190868.57,0.014682,1.0,0.333333,1.0105,-30,0.025907,117418.63,0.010556,108164.92,0.009833,299033.49,0.014587
2123,737,0.232933,CYRELA,1,7761,0.781571,1,1.0,58,0.025174,2019-03-30,1.000000,0.250000,1.0095,2022-06-01,0.033333,0.500000,1.0110,851966,0.065536,851965.56,0.065536,1.0,0.333333,1.0105,-29,0.025043,236023.62,0.021218,211029.91,0.019185,1062995.47,0.051853
1853,2067,0.653287,CYRELA,1,8312,0.837059,1,1.0,506,0.219618,2020-09-30,1.000000,0.750000,1.0100,2023-07-01,0.033333,0.583333,1.0115,164842,0.012680,164841.78,0.012680,1.0,0.333333,1.0105,-20,0.017271,211962.24,0.019055,196900.40,0.017900,361742.18,0.017646
1217,1859,0.587547,CYRELA,1,3259,0.328197,1,1.0,92,0.039931,2020-11-24,0.800000,0.916667,1.0100,2023-10-01,0.033333,0.833333,1.0115,562785,0.043291,562785.12,0.043291,1.0,0.333333,1.0105,-945,0.816062,329573.10,0.029628,309008.00,0.028092,871793.12,0.042526


In [6]:
df_train.describe()

,empresa,p_empresa,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
count,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2248.0,2248.0,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2248.0,2.248000e+03,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000
mean,1792.344751,0.566481,1.686833,7170.653915,0.722120,1.0,1.0,428.220196,0.185859,0.786877,0.599310,1.009744,3.333333e-02,0.578218,1.011114,6.382846e+05,0.049099,6.411160e+05,0.049317,1.0,3.333333e-01,1.010500e+00,-78.161922,0.067497,3.140547e+05,0.028233,2.819956e+05,0.025636,9.231116e+05,0.045030
std,845.660775,0.267276,0.752841,2431.498928,0.244864,0.0,0.0,491.714023,0.213418,0.252600,0.296702,0.000519,7.495673e-16,0.272753,0.000418,7.836957e+05,0.060284,7.774622e+05,0.059805,0.0,6.829391e-15,2.176521e-14,153.090383,0.132202,5.266069e+05,0.047341,4.537171e+05,0.041247,1.041685e+06,0.050814
min,164.000000,0.051833,1.000000,2339.000000,0.235549,1.0,1.0,1.000000,0.000434,0.100000,0.083333,1.004500,3.333333e-02,0.083333,1.006000,2.160000e+02,0.000017,2.160800e+02,0.000017,1.0,3.333333e-01,1.010500e+00,-1158.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,5.000000e+04,0.002439
25%,1166.000000,0.368521,1.000000,7470.000000,0.752266,1.0,1.0,102.000000,0.044271,0.600000,0.333333,1.009500,3.333333e-02,0.416667,1.011000,2.230878e+05,0.017161,2.243867e+05,0.017261,1.0,3.333333e-01,1.010500e+00,-52.000000,0.017271,6.208592e+04,0.005581,5.797075e+04,0.005270,3.682429e+05,0.017963
50%,2043.500000,0.645860,2.000000,7848.000000,0.790332,1.0,1.0,191.000000,0.082899,0.900000,0.666667,1.010000,3.333333e-02,0.583333,1.011000,4.082130e+05,0.031401,4.100624e+05,0.031543,1.0,3.333333e-01,1.010500e+00,-32.000000,0.027634,1.600377e+05,0.014387,1.474997e+05,0.013409,5.961705e+05,0.029081
75%,2154.000000,0.680784,2.000000,8722.000000,0.878348,1.0,1.0,618.250000,0.268338,1.000000,0.916667,1.010000,3.333333e-02,0.833333,1.011500,7.710162e+05,0.059309,7.764348e+05,0.059726,1.0,3.333333e-01,1.010500e+00,-20.000000,0.044905,3.800577e+05,0.034166,3.427128e+05,0.031156,1.206142e+06,0.058836
max,3164.000000,1.000000,3.000000,9930.000000,1.000000,1.0,1.0,2221.000000,0.963976,1.033333,1.000000,1.010500,3.333333e-02,1.000000,1.012000,1.300000e+07,1.000000,1.300000e+07,1.000000,1.0,3.333333e-01,1.010500e+00,0.000000,1.000000,1.100000e+07,0.988875,1.100000e+07,1.000000,2.050000e+07,1.000000


## Categorias de Atraso

Para a massa de dados disponível, optamos pela seguinte divisão de categorias:

1.   **Atraso até 30 dias** (*983* registros para treino).
2.   **Atraso até 90 dias** (*884* registros para treino).
3.   **Atraso superior 90 dias** (*381* registros para treino).



In [7]:
y_train.value_counts()

0    983
1    884
2    381
Name: p_dias_atraso_category, dtype: int64

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

## Atributos Utilizados no Treino

*   *p_empresa*
*   *p_marca*
*   *p_obra*
*   *p_unidade*
*   *p_saldo_devedor*
*   *p_valor_pago*
*   *p_vgv*
*   *p_dt_venda_year*
*   *p_dt_chaves_year*

Totalizando **9 atributos** utilizados no treinamento do modelo.


In [9]:
NUMERIC_COLUMNS = ['p_empresa', 'p_marca', 'p_obra', 'p_unidade',
                   'p_saldo_devedor', 'p_valor_pago', 'p_vgv',
                   'p_dt_venda_year', 'p_dt_chaves_year']

feature_columns = []

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

## Definição do Classificador

Após alguns testes, os melhores parâmetros do modelo, foram os seguintes:

*   Duas camadas ocultas (9 unidades cada).
*   Otimizador Adam.



In [10]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    optimizer='Adam',
    hidden_units=[9, 9],
    n_classes=3)

## Treino

O melhor número de iterações nos treinos (tempo X perda), foi **10.000** iterações. Importante não treinar demais o modelo (depende do volume de dados disponível), afim de evitar o *overfitting*.

In [11]:
classifier.train(
    input_fn=lambda: input_fn(df_train, y_train, training=True),
    steps=10000)

## Validação do Modelo

Utiliza-se os *20%* dos dados, não utilizados no treino, e, portanto, não vistos pelo modelo, para a validação de acurácia.

In [12]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(df_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-22T02:07:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8jb98ssg/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28780s
INFO:tensorflow:Finished evaluation at 2021-06-22-02:07:35
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.48, average_loss = 0.94754946, global_step = 10000, loss = 0.94734925
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp8jb98ssg/model.ckpt-10000

Test set accuracy: 0.480



## Conclusão

Após alguns testes, a acurácia geral do modelo, oscilou entre **40 e 50%**, o que consideramos muito baixo. Sugerimos o aumento no volume de dados (algo em torno de 10.000 registros), e a ampliação do escopo, ou seja, que haja maior cruzamento de dados (demais fontes), afim do aumento da acurácia do modelo, com isso, haverá maior facilidade em detecção de correlações.

---

